In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(
    "india_ooh_traffic_audience_5M.csv",
    engine="pyarrow",
    dtype={
        "city": "category",
        "state": "category",
        "metro_tier": "category",
        "area_type": "category",
        "income_segment": "category",
        "avg_daily_traffic": "int32",
        "car_count": "int32",
        "bike_count": "int32",
        "bus_count": "int32",
        "truck_count": "int32",
        "pedestrian_daily": "int32",
        "pct_age_15_24": "float32",
        "pct_age_25_44": "float32",
        "pct_age_45_60": "float32",
        "pct_age_60_plus": "float32",
        "working_prof_share": "float32",
        "student_share": "float32",
        "blue_collar_share": "float32",
        "num_schools_1km": "int16",
        "num_offices_1km": "int16",
        "num_malls_1km": "int16",
        "num_hospitals_1km": "int16",
        "num_factories_3km": "int16",
    }
)



In [5]:
df.head()

,id,city,state,metro_tier,area_type,avg_daily_traffic,car_count,bike_count,bus_count,truck_count,...,student_share,blue_collar_share,num_schools_1km,num_offices_1km,num_malls_1km,num_hospitals_1km,num_factories_3km,youth_index,affluence_index,primary_audience_tag
0,IN0000001,Pune,Maharashtra,2,Residential,79529,16618,32679,28826,1404,...,0.205791,0.107153,5,5,1,0,0,49.733642,20,Families/Residents
1,IN0000002,Hyderabad,Telangana,1,Industrial,91096,39335,48068,3597,94,...,0.076614,0.469256,0,6,3,1,5,27.944351,60,IndustrialWorkers
2,IN0000003,Nagpur,Maharashtra,2,Industrial,66729,22687,41985,1656,399,...,0.101347,0.503964,0,0,1,1,15,9.447267,35,IndustrialWorkers
3,IN0000004,Visakhapatnam,Andhra Pradesh,2,Residential,58823,21352,31559,740,5170,...,0.391002,0.148561,2,4,2,3,0,24.302821,35,Students/Youth
4,IN0000005,Surat,Gujarat,2,Highway,95897,55332,38357,1678,528,...,0.023291,0.229422,2,1,1,1,2,16.156703,90,WorkingProfessionals


In [7]:
# 2. Ensure percentage columns are 0–1
# -------------------------
percent_cols = [
    "pct_age_15_24",
    "pct_age_25_44",
    "pct_age_45_60",
    "pct_age_60_plus",
    "pct_female",
    "working_prof_share",
    "student_share",
    "blue_collar_share",
    "morning_peak_share",
    "evening_peak_share",
    "offpeak_share"
]

for col in percent_cols:
    if col in df.columns:
        # If values look like 0–100, convert to 0–1
        if df[col].max() > 1.5:   # >1.5 is a safe guard
            df[col] = df[col] / 100.0


In [8]:
# 3. Helper: safe min-max scaling 0–1
# -------------------------
def min_max(series):
    s = series.astype(float)
    min_v = s.min()
    max_v = s.max()
    if pd.isna(min_v) or pd.isna(max_v) or min_v == max_v:
        # no variation → return 0.5 for non-null rows
        return pd.Series(np.where(s.notna(), 0.5, np.nan), index=s.index)
    return (s - min_v) / (max_v - min_v)

In [11]:
# 4. Create some derived features
# -------------------------

# 4.1 Vehicle totals & mix
df["total_vehicles"] = (
    df[["car_count", "bike_count", "bus_count", "truck_count"]].sum(axis=1)
)

# Avoid division by zero
den = df["total_vehicles"].replace(0, np.nan)

df["car_share"]  = df["car_count"]  / den
df["bike_share"] = df["bike_count"] / den
df["bus_share"]  = df["bus_count"]  / den
df["truck_share"]= df["truck_count"] / den

# 4.2 Income segment numeric mapping
income_map = {
    "Low": 1,
    "LowerMiddle": 2,
    "Middle": 3,
    "UpperMiddle": 4,
    "Affluent": 5
}
df["income_segment_num"] = df["income_segment"].map(income_map)

df['income_segment_num'] = df['income_segment_num'].astype(int)

# If some rows are missing, fill with median (numeric, so median works)
df["income_segment_num"] = df["income_segment_num"].fillna(
    df["income_segment_num"].median()
)




In [12]:
# -------------------------
# 5. Normalize inputs for indices
# -------------------------
df["traffic_norm"]          = min_max(df["avg_daily_traffic"])
df["offices_norm"]          = min_max(df["num_offices_1km"])
df["schools_norm"]          = min_max(df["num_schools_1km"])
df["malls_norm"]            = min_max(df["num_malls_1km"])
df["factories_norm"]        = min_max(df["num_factories_3km"])
df["pedestrian_norm"]       = min_max(df["pedestrian_daily"])
df["income_norm"]           = min_max(df["income_segment_num"])
df["car_share_norm"]        = min_max(df["car_share"])
df["bus_share_norm"]        = min_max(df["bus_share"])
df["truck_share_norm"]      = min_max(df["truck_share"])
df["blue_collar_share_norm"]= min_max(df["blue_collar_share"])

# ensures NAs for missing vehicles don’t break logic
df[["car_share_norm","bus_share_norm","truck_share_norm"]] = df[
    ["car_share_norm","bus_share_norm","truck_share_norm"]
].fillna(0.0)

In [13]:
# 6. Build indices (0–100)
# -------------------------

# 6.1 WorkingProIndex:
#    - working_prof_share (who)
#    - commute_vehicle_mix: cars + buses (how they travel)
#    - offices nearby (where they work)
#    - traffic volume (how many people)
df["commute_vehicle_mix"] = (
    0.7 * df["car_share_norm"] + 0.3 * df["bus_share_norm"]
)

df["WorkingProIndex_raw"] = (
    0.4 * df["working_prof_share"] +
    0.3 * df["commute_vehicle_mix"] +
    0.2 * df["offices_norm"] +
    0.1 * df["traffic_norm"]
)

df["WorkingProIndex"] = min_max(df["WorkingProIndex_raw"]) * 100.0

# 6.2 StudentIndex:
#    - student_share + youth age bracket
#    - schools nearby
#    - pedestrian traffic
#    - evening_peak_share (hanging out after classes)
df["StudentIndex_raw"] = (
    0.35 * df["student_share"] +
    0.25 * df["pct_age_15_24"] +
    0.2  * df["schools_norm"] +
    0.1  * df["pedestrian_norm"] +
    0.1  * df["evening_peak_share"]
)

df["StudentIndex"] = min_max(df["StudentIndex_raw"]) * 100.0

# 6.3 AffluenceIndex:
#    - income_segment
#    - malls + offices
#    - car ownership (cars are a proxy for affluence)
#    - core earning age (25–44)
df["AffluenceIndex_raw"] = (
    0.35 * df["income_norm"] +
    0.25 * df["malls_norm"] +
    0.15 * df["offices_norm"] +
    0.15 * df["car_share_norm"] +
    0.10 * df["pct_age_25_44"]
)

df["AffluenceIndex"] = min_max(df["AffluenceIndex_raw"]) * 100.0

# 6.4 Optional: BlueCollarIndex for your heatmaps
#    - blue_collar_share
#    - factories nearby
#    - trucks
#    - off-peak share (night shifts, etc.)
df["BlueCollarIndex_raw"] = (
    0.4 * df["blue_collar_share"] +
    0.25 * df["factories_norm"] +
    0.2  * df["truck_share_norm"] +
    0.15 * df["offpeak_share"]
)

df["BlueCollarIndex"] = min_max(df["BlueCollarIndex_raw"]) * 100.0


In [14]:
# 7. Rank & segment sites
# -------------------------

# 7.1 Percentile ranks for each index
df["WorkingProIndex_pct"]   = df["WorkingProIndex"].rank(pct=True)
df["StudentIndex_pct"]      = df["StudentIndex"].rank(pct=True)
df["AffluenceIndex_pct"]    = df["AffluenceIndex"].rank(pct=True)
df["BlueCollarIndex_pct"]   = df["BlueCollarIndex"].rank(pct=True)

# 7.2 Elite / Top segments
df["is_elite_workingpro"] = np.where(df["WorkingProIndex_pct"] >= 0.95, "Elite WorkingPro", "No")
df["is_top_youth"]        = np.where(df["StudentIndex_pct"] >= 0.90, "Top Youth", "No")
df["is_top_affluent"]     = np.where(df["AffluenceIndex_pct"] >= 0.90, "Top Affluent", "No")
df["is_top_bluecollar"]   = np.where(df["BlueCollarIndex_pct"] >= 0.90, "Top BlueCollar", "No")

# 7.3 Balanced vs Hyper-skewed segments
def dominant_segment(row, threshold_ratio=1.3):
    # find maximum index among the four
    indices = {
        "WorkingPro": row["WorkingProIndex"],
        "Youth":      row["StudentIndex"],
        "Affluent":   row["AffluenceIndex"],
        "BlueCollar": row["BlueCollarIndex"]
    }
    max_seg = max(indices, key=indices.get)
    max_val = indices[max_seg]

    # second highest
    sorted_vals = sorted(indices.values(), reverse=True)
    second_val = sorted_vals[1] if len(sorted_vals) > 1 else 0

    if second_val == 0:
        return "Hyper-" + max_seg

    if max_val >= threshold_ratio * second_val:
        return "Hyper-" + max_seg
    else:
        return "Balanced"

df["audience_balance_tag"] = df.apply(dominant_segment, axis=1)

# 7.4 Primary audience tag (if you want a simple label for Power BI)
conditions = [
    df["WorkingProIndex"] >= df[["StudentIndex", "AffluenceIndex", "BlueCollarIndex"]].max(axis=1),
    df["StudentIndex"]    >= df[["WorkingProIndex", "AffluenceIndex", "BlueCollarIndex"]].max(axis=1),
    df["AffluenceIndex"]  >= df[["WorkingProIndex", "StudentIndex", "BlueCollarIndex"]].max(axis=1),
    df["BlueCollarIndex"] >= df[["WorkingProIndex", "StudentIndex", "AffluenceIndex"]].max(axis=1)
]
choices = ["WorkingPro-led", "Youth-led", "Affluent-led", "BlueCollar-led"]
df["primary_audience_tag_new"] = np.select(conditions, choices, default="Mixed")

In [16]:
# 8. Export to CSV
# -------------------------
out_cols = [
    "id", "city", "state", "metro_tier", "area_type",
    "avg_daily_traffic", "pedestrian_daily",
    "income_segment",
    "working_prof_share", "student_share", "blue_collar_share",
    "num_schools_1km", "num_offices_1km", "num_malls_1km", "num_factories_3km",
    "WorkingProIndex", "StudentIndex", "AffluenceIndex", "BlueCollarIndex",
    "is_elite_workingpro", "is_top_youth", "is_top_affluent", "is_top_bluecollar",
    "audience_balance_tag", "primary_audience_tag_new"
]


df_top500 = df.sort_values(
    ["WorkingProIndex", "AffluenceIndex"],  # sort by both
    ascending=False
).head(500)

df_top500[out_cols].to_csv("sites_top500.csv", index=False)
print("Exported top 500 high-value sites")

Exported top 500 high-value sites
